In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import hashlib
from glob import glob
import os
import random
import shutil

In [32]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

In [2]:
DATASET_PATH= "./data/images"
!ls {DATASET_PATH}

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

# Create Training and Validation sets

In [14]:
IMDIR = './data/images/'
os.mkdir('./data/train')
os.mkdir('./data/valid')
for i in tqdm(os.listdir(IMDIR)):
    train_path = f'./data/train/{i}/'
    valid_path = f'./data/valid/{i}/'
    os.makedirs(train_path,exist_ok=True)
    os.makedirs(valid_path,exist_ok=True)
    ims = pd.Series(os.listdir(f'{IMDIR}/{i}'))
    val_idxs = random.sample(list(ims.index), round(.2*len(ims)))
    for j in ims.index:
        if j in val_idxs:
            shutil.copy(f'{IMDIR}/{i}/{ims[j]}', valid_path)
        else:
            shutil.copy(f'{IMDIR}/{i}/{ims[j]}', train_path)

In [44]:
len(os.listdir(f'data/train/{i}'))

800